In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

driving_log_filename = './data/driving_log.csv'

df = pd.read_csv(driving_log_filename)
df.head()

In [ ]:
df.loc[:10]

In [ ]:
len(df)

In [ ]:
def batch_generator(df, batch_size):
    while True:
        df2 = df.sample(frac=1)
        start, end = 0, batch_size

        while end<=len(df):
            sub = df2[start:end]
            y_batch = sub['steering'].as_matrix()
            X_batch = []

            cameras = sub[['center', 'left', 'right']]
            for i in range(len(sub)):
                img = [np.asarray(Image.open('./data/'+ fn.strip())) for fn in list(cameras.iloc[i])]
                X_batch.append(np.concatenate(img,2))

            start += batch_size
            end += batch_size

            yield np.asarray(X_batch), y_batch


    

In [ ]:
X, y = None, None
for XX, yy in batch_generator(df, 2):
    X, y = XX, yy
    break

In [ ]:
# import csv
# import os.path
# import pickle

# if not os.path.isfile(dataname):
#     X = []
#     y = []
#     with open('./data/driving_log.csv', 'r') as csvfile:
#         reader = csv.reader(csvfile, skipinitialspace=True)
#         headers = next(reader)
#         count = 0
#         for row in reader:
#             img = [np.asarray(Image.open('./data/' + fn.strip()).resize((80,40), Image.ANTIALIAS)) for fn in row[:3]]
#             X.append(np.concatenate(img,2))
#             y.append(float(row[3]))
    
#     X = np.array(X)
#     y = np.array(y)
    
#     with open(dataname, 'wb') as handle:
#         pickle.dump([X, y], handle, protocol = pickle.HIGHEST_PROTOCOL)
        
# else:
#     with open(dataname, 'rb') as handle:
#         X, y = pickle.load(handle)    
            

In [ ]:
# X.shape

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Activation, Dense, Dropout, Lambda
from keras.activations import relu, softmax
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=X.shape[1:]))
model.add(Convolution2D(64, 5, 5, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))

In [ ]:
val_ratio = 0.2
train_mask = np.random.rand(len(df)) < 1-val_ratio
df_train = df[train_mask]
df_val = df[~train_mask]

In [ ]:
model.compile('adam', 'mean_squared_error', ['mean_absolute_error'])
batch_size = 128
samples_per_epoch = (len(df_train)+batch_size-1)/batch_size
history = model.fit_generator(batch_generator(df_train, batch_size), samples_per_epoch, nb_epoch=2, 
                              validation_data=batch_generator(df_val, batch_size),nb_val_samples=len(df_val))


In [ ]:
with open("model.json", "w") as jsonfile:
    jsonfile.write(model.to_json())
model.save_weights("model.h5")